In [ ]:
# For DS-1801
# Preconditions: either tensor_model_parallel=false or _DATA_PARALLEL_GROUP_YUXUAN_RANK, UNEQUAL and _MODEL_PARALLEL_GROUP_YUXUAN_RANK, CONSISTENT.
import json
import collections
import ast
import torch.nn.modules.module
counter = {}

with open('proxy_log.json', 'r') as f:
    for line in f:
        trace = json.loads(line)
        # print(trace)
        if 'value' in trace:
            # add other fields as needed
            if (trace['value'], trace['var_name']) not in counter:
                counter[(trace['value'], trace['var_name'])] = []
            meta_vars_str = json.loads(trace['meta_vars'])  # Convert meta_vars to a dictionary
            meta_vars_str = meta_vars_str.strip('"')

            # Convert the string to a dictionary
            meta_vars = ast.literal_eval(meta_vars_str)
            # print((meta_vars))
            if 'tensor_model_parallel' in trace['attributes'] and '_DATA_PARALLEL_GROUP_YUXUAN_RANK' in meta_vars and "_MODEL_PARALLEL_GROUP_YUXUAN_RANK" in meta_vars:
                tensor_model_parallel = trace['attributes']['tensor_model_parallel']
                data_parallel_group = meta_vars['_DATA_PARALLEL_GROUP_YUXUAN_RANK']
                model_parallel_group = meta_vars['_MODEL_PARALLEL_GROUP_YUXUAN_RANK']
                step = meta_vars['step']
                counter[(trace['value'], trace['var_name'])].append({
                    'tensor_model_parallel': tensor_model_parallel,
                    'data_parallel_group': data_parallel_group,
                    'model_parallel_group': model_parallel_group,
                    'step': step
                })

# Print variable names and their counts, sorted by count in descending order
for value, attr in counter.items():
    verify_dict = {}
    for i in attr:
        if 'tensor_model_parallel' in i and 'data_parallel_group' in i and 'model_parallel_group' in i:
            if i['tensor_model_parallel'] == True:
                if verify_dict.get((i['data_parallel_group'], i['model_parallel_group'])) is None:
                    verify_dict[(i['data_parallel_group'], i['model_parallel_group'])] = i['step']
                else:
                    print(f"data_parallel_group: {i['data_parallel_group']}, model_parallel_group: {i['model_parallel_group']}")
                    print(f"Duplicate tensor_model_parallel for {value}")
                    print(f"step: {i['step']}")
                    print(f"previous step: {verify_dict[(i['data_parallel_group'], i['model_parallel_group'])]}")
                    verify_dict[(i['data_parallel_group'], i['model_parallel_group'])] = i['step']
    print(f'{value}: {attr}')